In [1]:

import requests
import pandas as pd
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes
#=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [104]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_Hyderabad_Corporation_wards").text

In [105]:

soup = BeautifulSoup(url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Hyderabad Corporation wards - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Hyderabad_Corporation_wards","wgTitle":"List of Hyderabad Corporation wards","wgCurRevisionId":889213828,"wgRevisionId":889213828,"wgArticleId":22711399,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from February 2019","Use Indian English from February 2019","All Wikipedia articles written in Indian English","Hyderabad, India-related lists","Administrative divisions of India by city","Greater Hyderabad Municipal Corporation"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext"

In [106]:
table = soup.find('table')
table

<table class="wikitable">
<tbody><tr>
<th>Zone
</th>
<th>Circle
</th>
<th>Ward
</th></tr>
<tr>
<th rowspan="23"><a class="mw-redirect" href="/wiki/L._B._Nagar" title="L. B. Nagar">L. B. Nagar</a> (East Zone)
</th>
<td rowspan="6">1. <a href="/wiki/Kapra" title="Kapra">Kapra</a>
</td>
<td>1. <a href="/wiki/Kapra" title="Kapra">Kapra</a>
</td></tr>
<tr>
<td>2. <a href="/wiki/A._S._Rao_Nagar" title="A. S. Rao Nagar">Dr AS Rao Nagar</a> East Zone
</td></tr>
<tr>
<td>3. <a class="new" href="/w/index.php?title=Cherlapalli&amp;action=edit&amp;redlink=1" title="Cherlapalli (page does not exist)">Cherlapalli</a>
</td></tr>
<tr>
<td>4. <a class="new" href="/w/index.php?title=Meerpet_HB_Colony&amp;action=edit&amp;redlink=1" title="Meerpet HB Colony (page does not exist)">Meerpet HB Colony</a>
</td></tr>
<tr>
<td>5. <a href="/wiki/Mallapur" title="Mallapur">Mallapur</a>
</td></tr>
<tr>
<td>6. <a href="/wiki/Nacharam" title="Nacharam">Nacharam</a>
</td></tr>
<tr>
<td rowspan="4">2. <a href="/wiki/U

In [107]:
links = table.findAll('a')
links

[<a class="mw-redirect" href="/wiki/L._B._Nagar" title="L. B. Nagar">L. B. Nagar</a>,
 <a href="/wiki/Kapra" title="Kapra">Kapra</a>,
 <a href="/wiki/Kapra" title="Kapra">Kapra</a>,
 <a href="/wiki/A._S._Rao_Nagar" title="A. S. Rao Nagar">Dr AS Rao Nagar</a>,
 <a class="new" href="/w/index.php?title=Cherlapalli&amp;action=edit&amp;redlink=1" title="Cherlapalli (page does not exist)">Cherlapalli</a>,
 <a class="new" href="/w/index.php?title=Meerpet_HB_Colony&amp;action=edit&amp;redlink=1" title="Meerpet HB Colony (page does not exist)">Meerpet HB Colony</a>,
 <a href="/wiki/Mallapur" title="Mallapur">Mallapur</a>,
 <a href="/wiki/Nacharam" title="Nacharam">Nacharam</a>,
 <a href="/wiki/Uppal_Kalan" title="Uppal Kalan">Uppal</a>,
 <a class="new" href="/w/index.php?title=Chilukanagar&amp;action=edit&amp;redlink=1" title="Chilukanagar (page does not exist)">Chilukanagar</a>,
 <a href="/wiki/Habsiguda" title="Habsiguda">Habsiguda</a>,
 <a href="/wiki/Ramanthapur" title="Ramanthapur">Ramanth

In [108]:
Localities = []
for link in links:
    Localities.append(link.get('title'))
    print(Localities)

['L. B. Nagar']
['L. B. Nagar', 'Kapra']
['L. B. Nagar', 'Kapra', 'Kapra']
['L. B. Nagar', 'Kapra', 'Kapra', 'A. S. Rao Nagar']
['L. B. Nagar', 'Kapra', 'Kapra', 'A. S. Rao Nagar', 'Cherlapalli (page does not exist)']
['L. B. Nagar', 'Kapra', 'Kapra', 'A. S. Rao Nagar', 'Cherlapalli (page does not exist)', 'Meerpet HB Colony (page does not exist)']
['L. B. Nagar', 'Kapra', 'Kapra', 'A. S. Rao Nagar', 'Cherlapalli (page does not exist)', 'Meerpet HB Colony (page does not exist)', 'Mallapur']
['L. B. Nagar', 'Kapra', 'Kapra', 'A. S. Rao Nagar', 'Cherlapalli (page does not exist)', 'Meerpet HB Colony (page does not exist)', 'Mallapur', 'Nacharam']
['L. B. Nagar', 'Kapra', 'Kapra', 'A. S. Rao Nagar', 'Cherlapalli (page does not exist)', 'Meerpet HB Colony (page does not exist)', 'Mallapur', 'Nacharam', 'Uppal Kalan']
['L. B. Nagar', 'Kapra', 'Kapra', 'A. S. Rao Nagar', 'Cherlapalli (page does not exist)', 'Meerpet HB Colony (page does not exist)', 'Mallapur', 'Nacharam', 'Uppal Kalan', 'Ch

In [109]:
del Localities[0:2]
Localities

['Kapra',
 'A. S. Rao Nagar',
 'Cherlapalli (page does not exist)',
 'Meerpet HB Colony (page does not exist)',
 'Mallapur',
 'Nacharam',
 'Uppal Kalan',
 'Chilukanagar (page does not exist)',
 'Habsiguda',
 'Ramanthapur',
 'Uppal Village',
 'Hayathnagar',
 'Nagole',
 'Mansoorabad, Hyderabad (page does not exist)',
 'Hayathnagar',
 'BN Reddy Nagar (page does not exist)',
 'LB Nagar',
 'Vanasthalipuram',
 'Hasthinapuram (page does not exist)',
 'Champapet',
 'Lingojiguda',
 'Saroornagar',
 'Saroornagar',
 'Rama Krishna Puram, Hyderabad (page does not exist)',
 'Kothapet, Hyderabad',
 'Chaitanyapuri (page does not exist)',
 'Gaddiannaram',
 'Charminar',
 'Malakpet',
 'Saidabad',
 'Moosrambagh',
 'Old Malakpet',
 'Akberbagh (page does not exist)',
 'Azampura',
 'Chawani (page does not exist)',
 'Dabeerpura',
 'Santhoshnagar (page does not exist)',
 'Rein Bazar',
 'Talabchanchalam (page does not exist)',
 'Gowlipura (page does not exist)',
 'Kurmaguda (page does not exist)',
 'IS SADAN (pa

In [117]:
df = pd.DataFrame(Localities)
df.head(20)

,0
0,Kapra
1,A. S. Rao Nagar
2,Cherlapalli (page does not exist)
3,Meerpet HB Colony (page does not exist)
4,Mallapur
5,Nacharam
6,Uppal Kalan
7,Chilukanagar (page does not exist)
8,Habsiguda
9,Ramanthapur


In [131]:
df1 = pd.read_csv("loca.csv")
df1.head()

,Unnamed: 0,Full Address,Latitude,Longitude
0,NaN,Kapra,17.488795,78.571667
1,NaN,A. S. Rao Nagar,17.478478,78.550637
2,NaN,Cherlapalli,17.468112,78.597364
3,NaN,Meerpet HB Colony,17.453117,78.564488
4,NaN,Mallapur,17.446466,78.574574


In [132]:
df1.drop('Unnamed: 0',inplace=True,axis=1)

In [134]:
df1.head(20)

,Full Address,Latitude,Longitude
0,Kapra,17.488795,78.571667
1,A. S. Rao Nagar,17.478478,78.550637
2,Cherlapalli,17.468112,78.597364
3,Meerpet HB Colony,17.453117,78.564488
4,Mallapur,17.446466,78.574574
5,Nacharam,17.824661,78.540530
6,Uppal Kalan,17.398377,78.558265
7,Chilukanagar,17.421812,78.561472
8,Habsiguda,17.409627,78.544114
9,Ramanthapur,17.385277,78.527751


In [135]:
address = 'Hyderabad'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Hyd are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Hyd are 17.3616079, 78.4746286.


In [136]:
df3 = df1.copy()
df3

,Full Address,Latitude,Longitude
0,Kapra,17.488795,78.571667
1,A. S. Rao Nagar,17.478478,78.550637
2,Cherlapalli,17.468112,78.597364
3,Meerpet HB Colony,17.453117,78.564488
4,Mallapur,17.446466,78.574574
5,Nacharam,17.824661,78.540530
6,Uppal Kalan,17.398377,78.558265
7,Chilukanagar,17.421812,78.561472
8,Habsiguda,17.409627,78.544114
9,Ramanthapur,17.385277,78.527751


In [140]:
map_bogota = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(df3['Latitude'], df3['Longitude'],df1):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bogota)  
    
map_bogota

In [170]:
# function to repeat the exploring process to all the neighborhoods in Toronto
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Localidad', 
                  'Localidad Latitude', 
                  'Localidad Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [171]:


limit = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'KYFBAIN1FENNLZD0XPDWGMQKMBXSVNKEPHQCMOFRJD13ODTJ'
CLIENT_SECRET = 'RXTFT3IB4BO2CUBGZ1NZFJUMJZ1X0LLER0OYT1SCAVAIFSFO'
VERSION = '20190922'

In [175]:
hyd_foodcourt = getNearbyVenues(names=df1['Full Address'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1fa931735')
hyd_foodcourt.head() #FoodCourt

https://api.foursquare.com/v2/venues/search?&client_id=KYFBAIN1FENNLZD0XPDWGMQKMBXSVNKEPHQCMOFRJD13ODTJ&client_secret=RXTFT3IB4BO2CUBGZ1NZFJUMJZ1X0LLER0OYT1SCAVAIFSFO&v=20190922&ll=17.4887947,78.57166740000001&radius=1000&limit=500&categoryId=4bf58dd8d48988d1fa931735
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5d35af76348eba00230e6472'}, 'response': {}}


UnboundLocalError: local variable 'results' referenced before assignment

In [172]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venues, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df1['Full Address'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [159]:
hyd_office.shape


(1778, 7)

In [160]:
hyd_hostel = getNearbyVenues(names=df1['Full Address'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ee931735')
hyd_hostel.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rama Krishna Puram,28.566008,77.176744,Jugaad Hostels,28.565317,77.185655,Hostel
1,Jambagh,17.379400,78.476615,Grand Elite Hotel,17.387812,78.475426,Hostel
2,Gunfoundry,17.396183,78.477326,VJR Hostel,17.393493,78.481374,Hostel
3,Gunfoundry,17.396183,78.477326,Hotel Rajmata,17.392639,78.471001,Hostel
4,Gunfoundry,17.396183,78.477326,Grand Elite Hotel,17.387812,78.475426,Hostel


In [161]:
hyd_hostel.shape

(29, 7)